<a href="https://colab.research.google.com/github/pedroafleite/instacart-market-basket-analysis/blob/main/instacart_2_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.3.1/en_core_web_lg-2.3.1.tar.gz
#!pip install en_core_web_lg

In [ ]:
#pip install -U spacy

In [ ]:
import spacy.cli
spacy.cli.download("en_core_web_lg")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
import pandas as pd
import re, nltk, spacy, gensim
import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
aisles = pd.read_csv('/content/drive/MyDrive/instacart/aisles.csv')
departments = pd.read_csv('/content/drive/MyDrive/instacart/departments.csv')
order_products_prior = pd.read_csv('/content/drive/MyDrive/instacart/order_products__prior.csv')
order_products_train = pd.read_csv('/content/drive/MyDrive/instacart/order_products__train.csv')
orders = pd.read_csv('/content/drive/MyDrive/instacart/orders.csv')
products = pd.read_csv('/content/drive/MyDrive/instacart/products.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/instacart/sample_submission.csv')

In [ ]:
order1 = order_products_train.merge(products, how='inner')
order1 = order1.sort_values(by=['order_id','add_to_cart_order'])
order1.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,1,49302,1,1,Bulgarian Yogurt,120,16
8,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16
152,1,10246,3,0,Organic Celery Hearts,83,4
1214,1,49683,4,0,Cucumber Kirby,83,4
3627,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15


In [ ]:
order1.shape

(1384617, 7)

In [ ]:
# Yield successive n-sized 
# chunks from l. 
#def divide_chunks(l, n): 
      
    # looping till length l 
#    for i in range(0, len(l), n):  
#        yield l[i:i + n]

# How many elements each 
# list should have 
#n = int(len(order1)/100)
  
#x = list(divide_chunks(order1, n)) 

In [ ]:
#order2 = x[0]
order2.shape

(324344, 7)

In [ ]:
df = order1.merge(orders, how='outer')
df.drop(["add_to_cart_order", "reordered", "eval_set", "order_number", 
         "order_dow", "order_hour_of_day", "days_since_prior_order"], axis=1, inplace=True)
df.head(10)

,order_id,product_id,product_name,aisle_id,department_id,user_id
0,1,49302.0,Bulgarian Yogurt,120.0,16.0,112108
1,1,11109.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108.0,16.0,112108
2,1,10246.0,Organic Celery Hearts,83.0,4.0,112108
3,1,49683.0,Cucumber Kirby,83.0,4.0,112108
4,1,43633.0,Lightly Smoked Sardines in Olive Oil,95.0,15.0,112108
5,1,13176.0,Bag of Organic Bananas,24.0,4.0,112108
6,1,47209.0,Organic Hass Avocado,24.0,4.0,112108
7,1,22035.0,Organic Whole String Cheese,21.0,16.0,112108
8,36,39612.0,Grated Pecorino Romano Cheese,2.0,16.0,79431
9,36,19660.0,Spring Water,115.0,7.0,79431


In [ ]:
df.shape

(4674491, 6)

In [ ]:
df['user_id'].nunique()

206209

In [ ]:
df['order_id'].nunique() # more orders than users, as expected

3421083

In [ ]:
df.isna().sum()

order_id               0
product_id       3289874
product_name     3289874
aisle_id         3289874
department_id    3289874
user_id                0
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df.shape

(1384617, 6)

In [ ]:
df.to_csv('/content/drive/MyDrive/instacart/df.csv',encoding='utf-8', index=False)

In [ ]:
def sent_to_words(sentences):
    ''' Tokenisation of sentences without punctuation'''
    
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True Remove the punctuation
        

data_words = list(sent_to_words(df["product_name"]))
print(data_words[:1])

[['bulgarian', 'yogurt']]


In [ ]:
len(data_words)/5

276923.4

In [ ]:
# Yield successive n-sized 
# chunks from l. 
def divide_chunks(l, n): 
      
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n]

# How many elements each 
# list should have 
n = int(len(data_words)/3)
  
x = list(divide_chunks(data_words, n)) 

In [ ]:
# 
data_words = x[0]
len(data_words)

461539

In [ ]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """It keeps the lemma of the words (lemma is the uninflected form of a word)"""
    
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' 
                                   for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [ ]:
# You don't have permission in Colab to load the model with normal spacy usage  .
# Instead, import the model and load it directly:
#import en_core_web_lg
#nlp = en_core_web_lg.load()

In [ ]:
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

In [ ]:
# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:10])

['bulgarian', 'organic milk fat whole milk cottage cheese', 'organic celery heart', '', 'lightly smoke sardine olive oil', 'bag organic banana', '', 'organic whole string cheese', 'grated cheese', 'spring']


In [ ]:
data_lemmatized_pd = pd.DataFrame(data_lemmatized)
data_lemmatized_pd.to_csv('/content/drive/MyDrive/instacart/data_lemmatized.csv',encoding='utf-8', index=False)

In [ ]:
# Create our products categories using LDA algorithm 

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

n_features = 3000 
n_components = 5 # number of categories
n_top_words = 15


print("Extraction of tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')

tf = tf_vectorizer.fit_transform(data_lemmatized)


def print_top_words(model, feature_names, n_top_words):
    
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0).fit(tf)

print("\n The categories in the LDA model :")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Extraction of tf features for LDA...

 The categories in the LDA model :
Topic #0: organic baby original tomato lemon juice sweet onion salt carrot lime potato roast raspberry garlic
Topic #1: organic banana chocolate strawberry bag apple sauce blueberry bar extra butter peanut oil cookie food
Topic #2: free water large cream almond sparkle bread egg gluten milk unsweetened wheat fresh brown ice
Topic #3: milk yogurt fat natural coconut cracker bean reduce snack low greek vanilla cereal total plain
Topic #4: cheese chip chicken spinach cheddar white green red slice breast half shred rice roll pizza



In [ ]:
#Creation of the Categories variable giving the category of each product purchased
topics = lda.transform(tf)
df_topics = pd.DataFrame(topics)
list_topics = []

for index, row in df_topics.iterrows():
    column_line = str(row.idxmax(max(row)))
    list_topics.append(column_line)
    
df["Categories"] = pd.DataFrame(list_topics)

In [ ]:
df['Categories'].value_counts()

0    140083
1     93051
2     84057
4     73963
3     70385
Name: Categories, dtype: int64

In [ ]:
df.head()

,order_id,product_id,product_name,aisle_id,department_id,user_id,Categories,Category 1,Category 3,Category 0,Category 4,Category 2,Category 6,Category 5
0,1,49302.0,Bulgarian Yogurt,120.0,16.0,112108,0,1,0,0,0,0,0,0
1,1,11109.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108.0,16.0,112108,3,0,1,0,0,0,0,0
2,1,10246.0,Organic Celery Hearts,83.0,4.0,112108,0,0,0,1,0,0,0,0
3,1,49683.0,Cucumber Kirby,83.0,4.0,112108,0,0,0,1,0,0,0,0
4,1,43633.0,Lightly Smoked Sardines in Olive Oil,95.0,15.0,112108,4,1,0,0,0,0,0,0


In [ ]:
df.dropna(axis=0, subset=["Categories"], inplace=True)

In [ ]:
# One hot encoding on the variable Categories

for category in df['Categories'].unique():
    df["Category {}".format(category)] = df['Categories'].str.contains(category).apply(lambda x:1 if x else 0)

In [ ]:
df.to_csv('/content/drive/MyDrive/instacart/df.csv',encoding='utf-8', index=False)